In [1]:
%load_ext autoreload

%autoreload 2

This notebook demonstrates how to use the local optimization procedure based on the topological descriptors by Kozlov et al. and EMT, as it is implemented in NPL.

In [2]:
import Core.Nanoparticle as NP
import Core.GOSearch as GOS
from LocalOpt.LocalOptimization import local_optimization

import Core.EnergyCalculator as EC

from ase.visualize import view
import pickle


In [3]:
"""Create a list of particles with fixed composition and shape (truncated octahedron), but random ordering
and calculate the energy using EMT
"""

def create_octahedron_training_set(n_particles, height, trunc, stoichiometry):
    emt_calculator = EC.EMTCalculator(fmax=0.03, steps=1000)
    
    training_set = []
    for i in range(n_particles):
        p = NP.Nanoparticle()
        p.truncated_octahedron(height, trunc, stoichiometry)
        emt_calculator.compute_energy(p)
        training_set.append(p)
        
    return training_set

In [4]:
"""Create one randomly ordered start particle"""

def create_start_particle(height, trunc, stoichiometry):
    start_particle = NP.Nanoparticle()
    start_particle.truncated_octahedron(height, trunc, stoichiometry)
    return start_particle

In [5]:
"""Create the training set with 30 particles"""
stoichiometry={'Pt' : 55, 'Au' : 24}

training_set = create_octahedron_training_set(30, 5, 1, stoichiometry)

In [28]:
"""First we create an Object for a global optimization search and pass to it references to functions that we
want to use for optimizing (local_optimization) and for creating a start configuration (create_start_particle)."""

guided_MC_search = GOS.GuidedSearch(local_optimization, create_start_particle)

"""We then have it fit the topological descriptors to the training set we just created"""
symbols = list(stoichiometry.keys()) #['Pt', 'Au']
guided_MC_search.fit_energy_expression(training_set, symbols)

"""We start the optimization by calling the run_multiple simulations function. In this case we want to do two
runs (n_sim_runs = 2). We can pass parameters to both functions (local optmization & create start particle) that will
be used when the optimization is actually started. In this case we make sure, that the start particle has the same
shape as the particles in the training set."""

n_sim_runs = 2
args_start = [5, 1, stoichiometry] # height, trunc, composition -> parameters of the create_start_particle function
results, run_times = guided_MC_search.run_multiple_simulations(n_sim_runs, args_start=args_start)

[-2.46365762e-01  4.29589750e-02  9.64958440e-01  4.29732720e-01
 -7.04720610e-11 -1.42619482e-14 -2.44046952e-17  5.08400810e-15
 -7.32553560e-15  1.13374119e-29  9.93633282e-01  1.00976108e+00
 -1.86122230e-31  1.08379826e+00  4.19416133e-46  0.00000000e+00
  1.21013457e+00]
Coef symbol_a: Au
Run: 0
Runtime: 0.03734965401235968
Run: 1
Runtime: 0.036429041996598244


In [29]:
"""The results object will be a list of optimization runs. Each optimization run holds the final structure as well
as the energies and the step number of the considered configurations. Run times for every run will be returned as a
list"""
print('Structure : {}'.format(results[0][0]))
print(' ')
print('(Energy, step) pairs:')
print(results[0][1])
print(' ')
print('Runtime in s: {}'.format(run_times[0]))

Structure : <Core.Nanoparticle.Nanoparticle object at 0x7f213f75cee0>
 
(Energy, step) pairs:
[(28.65324643579614, 0), (28.312711680110855, 2), (28.026979693884506, 4), (27.740453132310453, 6), (27.578428549976756, 8), (27.2919019884027, 10), (27.033174099231967, 12), (26.77365163471353, 14), (26.594465820102265, 16), (26.46928819960223, 18), (26.41528000549099, 20), (26.29157260159834, 22), (26.237564407487106, 24), (26.237564407487106, 26)]
 
Runtime in s: 0.03734965401235968


In [30]:
"""Display the final structure of the first run"""
p = results[0][0]
view(p.get_ase_atoms(), viewer='x3d')

In [31]:
"""This is not the optimal solution. We can use the Basin Hopping instead of the local optimization to find it.
For this we have to import the run_basin_hopping function and pass some additional start parameters to the search
object"""

from BH.BasinHopping import run_basin_hopping

guided_MC_search = GOS.GuidedSearch(run_basin_hopping, create_start_particle)

"""Fit energy expression"""
symbols = list(stoichiometry.keys()) #['Pt', 'Au']
guided_MC_search.fit_energy_expression(training_set, symbols)

"""For the Basin Hopping we also have to pass the number of (not necessarily distinct) basins we want to expore 
(n_hopping_attempts) and how strong we want to perturbate the locally optimal solutions (n_exchanges). We can
pass them the same way as for the start particle. run_basin_hopping and local optimization also take different
parameters. Fortunately their function signatures are the same for the first three parameters, so we can reuse
the GuidedSearch class for both optmization functions. The remaining parameters can be passed simply as list, 
which should prove useful if the function signature changes at some point or one ones to implement a different
optimization procedure."""
n_hopping_attempts = 20
n_exchanges = 10
args_bh = [n_hopping_attempts, n_exchanges]

n_sim_runs = 2
args_start = [5, 1, stoichiometry] # height, trunc, composition -> parameters of the create_start_particle function

results, run_times = guided_MC_search.run_multiple_simulations(n_sim_runs, args_start=args_start, args_gm = args_bh)

[-2.46365762e-01  4.29589750e-02  9.64958440e-01  4.29732720e-01
 -7.04720610e-11 -1.42619482e-14 -2.44046952e-17  5.08400810e-15
 -7.32553560e-15  1.13374119e-29  9.93633282e-01  1.00976108e+00
 -1.86122230e-31  1.08379826e+00  4.19416133e-46  0.00000000e+00
  1.21013457e+00]
Coef symbol_a: Au
Run: 0
Energy after local_opt: 26.282, lowest 26.282
Lowest energy: 26.149
Runtime: 0.6366964560002089
Run: 1
Energy after local_opt: 26.319, lowest 26.319
Lowest energy: 26.149
Runtime: 0.6761802340624854


In [32]:
"""Now we should have the best solution for this system, both runs finished with the same energy"""
p = results[0][0]
view(p.get_ase_atoms(), viewer='x3d')